In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import Counter

# local imports
from prepare import *
from evaluate import *

### Read in initial datasets if needed

In [2]:
raw_train, raw_train_labels, raw_test, specs, sample = read_raw_csvs()

### Load compiled train/test datasets

In [3]:
train, test = load_and_prep(raw_train_labels)

### Start throwing model mud at the wall

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier, 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model.SGDClassifier

quick_acc()